# RCA fish species and Temperature site attributes
 * Add HUC 12 and elevation data to temperature sites
 * Add AWC species and Northern pike data to RCA
  * all life stages/categories
 * Elevation stats for all RCAs
   * Mean min max
   * Mean elevation of contributing area
 * Percent Glacier cover calculation along flow acc network
 * 
 


### Import modules and set environments


In [1]:
import arcpy, os, zipfile, requests, datetime, sys, time, traceback
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")



from datetime import datetime
today = datetime.now()
#Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)
path = os.getcwd() # temporary working directories and geodatabases will be created here
print (path)

06022020
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes


### Set variables
 * All input data are stored on the T: so no local data are needed
  * Temporary working directors are created in the current working directory listed above
    * Set path = "desired working folder" if necessary

In [59]:
import datetime

### Variable Names/identifiers ###

region = "Anchor" #region of interest - used to name temporary directories
outgdbname = region + ".gdb" #Name of output gdb
outdirname = region + "_Attributes_" #Location to save outputs
identifierfield = "reachid" #Name of field storing Catchment/RCA field in input stream layer ex. NHDPlus = "NHDPLusID", rca streams = "reachid"

### Network Data ###

instudy =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\Anchor_Source.gdb\\anch_studyarea" #Polygon defining study area, used to extract nlcd raster
instreams = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\Anchor_Source.gdb\\anch_rca_reaches" #Streams with RCA code
inrca =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\Anchor_Source.gdb\\anchor_rcas" #RCAs for region of interest
awc_events = "T:\\Aquatic\\KFHP\\AWC\\2018\\2018GDB_statewide.gdb\\awcEventArcs" # AWC events 
tempsites = "T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\Anchor_Source.gdb\\tempsites_anchor" # Temperature locations
huc12 = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\WBDHU12.shp" # HUC12 from NHD gdb

flowrast =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\AnchStar_StrBrn_ad8ContribArea.tif" #flow accumulation raster for region of interest 
flowdirrast =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\AnchStar_StrBrn_d8flowdir.tif" #flow direction raster for region of interest
dem = "T:\\Aquatic\\KFHP\\TauDEM_Outputs\\AnchStar\\Tau_out\\AnchStar_StrBrn_2Int.tif" #DEM converted to Integer that matches extent and cell size of flow dir
orig_streamrast = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Anchor\\Source_Data\\AnchStar_StrBrn_src.tif" #stream grid from synthetic stream network

identname = region + "_NLCD_Identity"#name of output Identity
tabtablename = region + "_NLCD_Tab_Int"#Name for output tabulate intersection table

#### Final product output location ####
networkgdb = r'T:\\Aquatic\\KFHP\\Geodatabases\\Anchor.gdb'# Location of shared geodatabase on network for final products

### Generated variables ###

streamdesc = arcpy.Describe(instreams)
streams_name = streamdesc.name
studydesc = arcpy.Describe(instudy)
study_name = studydesc.name
identname = streams_name + "_NLCD_Identity"#name of output Identity
tabtablename = streams_name + "_NLCD_Tab_Int"#Name for output tabulate intersection table

time = datetime.datetime.now()
print ("Variables set on", time)



Variables set on 2020-02-06 15:07:47.218434


### Create temporary folder and gdb

In [3]:
dirname = outdirname + str(time_stamp)
temp_dir = os.path.join(path,dirname)
ziploc = os.path.join(temp_dir, 'zips')
extractloc = os.path.join(temp_dir, 'extracts')

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    os.makedirs(ziploc)
    os.makedirs(extractloc)        


outcheck = os.path.join(temp_dir,outgdbname)

if os.path.exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not os.path.exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,outgdbname)
    print ('Output geodatabase created at',outcheck)
    outgdb = tempgdb.getOutput(0)

Output location already exists C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb


In [4]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


indata = []
copylist = []
indata = [instudy,instreams,inrca,tempsites,awc_events]

for fc in indata:
    
    desc = arcpy.Describe(fc)
    fieldlist = arcpy.ListFields(fc)
    
    copy = arcpy.CopyFeatures_management(fc,desc.name)
    desc2 = arcpy.Describe(copy)
    copypath = os.path.join (desc2.path, desc2.name)
    copylist.append(copypath)

print (copylist)

['C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anch_studyarea', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anch_rca_reaches', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anchor_rcas', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\tempsites_anchor', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\awcEventArcs']


### Set input variables from temporary copies

In [5]:
for path in copylist:
    
    if "studyarea" in path:      
        studycopy = path
        print (studycopy)
        print ("")
    
    elif "reaches" in path:
        reachcopy = path
        print (reachcopy)
        print ("")
        
    elif "rcas" in path:
        rcacopy = path
        print (rcacopy)
        print ("")
        
    elif "tempsites" in path:
        tempcopy = path
        print (tempcopy)
        print ("")
    
    elif "awc" in path:
        awccopy = path
        print (awccopy)
        print ("")
    

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\anch_studyarea

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\anch_rca_reaches

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\anchor_rcas

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\tempsites_anchor

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\awcEventArcs



## Begin Geospatial operations

### Join HUC12 data to RCA
     * Format RCAs
      * Drop all fields other than rca_id
      * Recalculate elevation (min-mean-max)
      * Recalculate contributing area
      

### Largest overlap Spatial Join
 * code from <a href = "https://www.arcgis.com/home/item.html?id=e9cccd343bf84916bda1910c31e5eab2">Largest Overlap</a>
 


In [6]:
# Main function, all functions run in SpatialJoinOverlapsCrossings
def SpatialJoinLargestOverlap(target_features, join_features, out_fc, keep_all, spatial_rel):
    if spatial_rel == "largest_overlap":
        # Calculate intersection between Target Feature and Join Features
        intersect = arcpy.analysis.Intersect([target_features, join_features], "in_memory/intersect", "ONLY_FID")
        # Find which Join Feature has the largest overlap with each Target Feature
        # Need to know the Target Features shape type, to know to read the SHAPE_AREA oR SHAPE_LENGTH property
        geom = "AREA" if arcpy.Describe(target_features).shapeType.lower() == "polygon" and arcpy.Describe(join_features).shapeType.lower() == "polygon" else "LENGTH"
        fields = ["FID_{0}".format(os.path.splitext(os.path.basename(target_features))[0]),
                  "FID_{0}".format(os.path.splitext(os.path.basename(join_features))[0]),
                  "SHAPE@{0}".format(geom)]
        overlap_dict = {}
        with arcpy.da.SearchCursor(intersect, fields) as scur:
            for row in scur:
                try:
                    if row[2] > overlap_dict[row[0]][1]:
                        overlap_dict[row[0]] = [row[1], row[2]]
                except:
                    overlap_dict[row[0]] = [row[1], row[2]]

        # Copy the target features and write the largest overlap join feature ID to each record
        # Set up all fields from the target features + ORIG_FID
        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(target_features)
        fieldmap = arcpy.FieldMap()
        fieldmap.addInputField(target_features, arcpy.Describe(target_features).OIDFieldName)
        fld = fieldmap.outputField
        fld.type, fld.name, fld.aliasName = "LONG", "ORIG_FID", "ORIG_FID"
        fieldmap.outputField = fld
        fieldmappings.addFieldMap(fieldmap)
        # Perform the copy
        arcpy.conversion.FeatureClassToFeatureClass(target_features, os.path.dirname(out_fc), os.path.basename(out_fc), "", fieldmappings)
        # Add a new field JOIN_FID to contain the fid of the join feature with the largest overlap
        arcpy.management.AddField(out_fc, "JOIN_FID", "LONG")
        # Calculate the JOIN_FID field
        with arcpy.da.UpdateCursor(out_fc, ["ORIG_FID", "JOIN_FID"]) as ucur:
            for row in ucur:
                try:
                    row[1] = overlap_dict[row[0]][0]
                    ucur.updateRow(row)
                except:
                    if not keep_all:
                        ucur.deleteRow()
        # Join all attributes from the join features to the output
        joinfields = [x.name for x in arcpy.ListFields(join_features) if not x.required]
        arcpy.management.JoinField(out_fc, "JOIN_FID", join_features, arcpy.Describe(join_features).OIDFieldName, joinfields)

In [7]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

hucjoinname = region + "_rca_huc12_largeover_sj"
huccjoin = SpatialJoinLargestOverlap(rcacopy, huc12, hucjoinname, keep_all = "TRUE", spatial_rel = "largest_overlap" ) #Code from ESRI 

joindrop = []

awcjoin = os.path.join(outgdb+ "\\" + hucjoinname)

for field in arcpy.ListFields(awcjoin):
    joindrop.append(field.name)

keepfields = ['OBJECTID', 'Shape', 'rca_id','Shape_Length', 'Shape_Area','HUC12','Name']
for field in keepfields:   
    joindrop.remove(field)

arcpy.DeleteField_management(awcjoin, joindrop)

for field in arcpy.ListFields(awcjoin):
    print (field.name)

OBJECTID
Shape
rca_id
Shape_Length
Shape_Area
HUC12
Name


### Remove all fields from reaches except reach id and shape fields

In [8]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

reachfields = []
for field in arcpy.ListFields(reachcopy):
    reachfields.append(field.name)

print (reachfields)
print ("")

keepfields = ['OBJECTID', 'Shape', 'reachid','Shape_Length']
for field in keepfields:   
    reachfields.remove(field)

arcpy.DeleteField_management(reachcopy, reachfields)

for field in arcpy.ListFields(reachcopy):
    print (field.name)

['OBJECTID', 'Shape', 'Shape_Leng', 'reachid', 'Z_Min', 'Z_Max', 'Z_Mean', 'SLength', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'length_m', 'slope_P', 'slope_D', 'Shape_Length']

OBJECTID
Shape
reachid
Shape_Length


### Fish data
 * Join 5 salmon species and with lifestage data to each RCA
  * Format needs to be 3 columns for each species Spec/Lstag
  
 * Identity awc with rca first then spatial join and concatenate or selectby and populate fields

### Identity awc with rca

In [9]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

awcidentname = region + "_awc_rca_IDENTITY"
awcclip = arcpy.Clip_analysis(awccopy,awcjoin,"tempclip")
awcident = arcpy.Identity_analysis(awcclip,awcjoin,awcidentname,"ALL")

print (awcident)

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\Anchor_awc_rca_IDENTITY


### Add fish species/life stage fields and delete join fields

In [10]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

lifestages = ['r','s','p']
species = ['K','CH','CO','P','S']
cols = []
for s in species:
    for l in lifestages:
        colname = str(s) + "_" + str(l)
        cols.append(colname)
print(cols)

for field in cols:
    arcpy.AddField_management(awcjoin,field,"SHORT") #add awcfields to rcas

deletefields = ['Join_Count','TARGET_FID','Join_Count_1','TARGET_FID_1','JOIN_FID']
arcpy.DeleteField_management(awcjoin,deletefields)

['K_r', 'K_s', 'K_p', 'CH_r', 'CH_s', 'CH_p', 'CO_r', 'CO_s', 'CO_p', 'P_r', 'P_s', 'P_p', 'S_r', 'S_s', 'S_p']


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\Anchor_rca_huc12_largeover_sj'>

### Query AWC events and calculate presence, spawning, and rearing attributes for RCAs from AWC/RCA identity
 * SPECIES
  * K: CHINOOK
  * CH: CHUM
  * CO: COHO
  * P: PINK
  * S: SOCKEYE
 
 
 * LIFE STAGE
  * r: rearing
  * s: spawning
  * p: presence - Presence has been calculated such that p = 1 where present or rearing or spawning as recorded in AWC

In [11]:
from time import strftime
start = datetime.datetime.now()

print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))

fields = ['K_r', 'K_s', 'K_p', 'CH_r', 'CH_s', 'CH_p', 'CO_r', 'CO_s', 'CO_p', 'P_r', 'P_s', 'P_p', 'S_r', 'S_s', 'S_p']

### Chinook Selections ###
# "SPECIES = 'K' And LSTAGE = 'r' OR SPECIES = 'K' And LSTAGE = 's' OR SPECIES = 'K' And LSTAGE = 'p'" # Modify presence to include all

akr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 'r'")
rkr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', akr)
arcpy.CalculateField_management(rkr,'K_r', 1)

aks = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 's'")
rks = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', aks)
arcpy.CalculateField_management(rks,'K_s', 1)

akp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 'r' OR SPECIES = 'K' And LSTAGE = 's' OR SPECIES = 'K' And LSTAGE = 'p'")
rkp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', akp)
arcpy.CalculateField_management(rkp,'K_p', 1)

### Chum Selections ###
achr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 'r'")
rchr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achr)
arcpy.CalculateField_management(rchr,'CH_r', 1)

achs = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 's'")
rchs = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achs)
arcpy.CalculateField_management(rchs,'CH_s', 1)

achp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 'r' OR SPECIES = 'CH' And LSTAGE = 's' OR SPECIES = 'CH' And LSTAGE = 'p'")
rchp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achp)
arcpy.CalculateField_management(rchp,'CH_p', 1)

### Coho Selection ###
acor = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 'r'")
rcor = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acor)
arcpy.CalculateField_management(rcor,'CO_r', 1)

acos = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 's'")
rcos = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acos)
arcpy.CalculateField_management(rcos,'CO_s', 1)

acop = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 'r' OR SPECIES = 'CO' And LSTAGE = 's' OR SPECIES = 'CO' And LSTAGE = 'p'")
rcop = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acop)
arcpy.CalculateField_management(rcop,'CO_p', 1)

### Pink Selection ###
apr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 'r'")
rpr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', apr)
arcpy.CalculateField_management(rpr,'P_r', 1)

aps = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 's'")
rps = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', aps)
arcpy.CalculateField_management(rps,'P_s', 1)

app = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 'r' OR SPECIES = 'P' And LSTAGE = 's' OR SPECIES = 'P' And LSTAGE = 'p'")
rpp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', app)
arcpy.CalculateField_management(rpp,'P_p', 1)

### Sockeye Selection ###
asr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 'r'")
rsr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', asr)
arcpy.CalculateField_management(rsr,'S_r', 1)

ass = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 's'")
rss = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', ass)
arcpy.CalculateField_management(rss,'S_s', 1)

asp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 'r' OR SPECIES = 'S' And LSTAGE = 's' OR SPECIES = 'S' And LSTAGE = 'p'")
rsp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', asp)
arcpy.CalculateField_management(rsp,'S_p', 1)

stop = datetime.datetime.now()
elapsed = stop - start
print( "End script: " + strftime("%Y-%m-%d %H:%M:%S") + " Runtime = " +  str(elapsed))


Start script: 2020-02-06 14:26:49
End script: 2020-02-06 14:27:00 Runtime = 0:00:10.740214


### Convert null values in fish species columns to zero

In [12]:
fieldList =  fields
with arcpy.da.UpdateCursor(awcjoin, fieldList) as cursor:
    fRange = range(len(fieldList)) # create an index 0 to the number of elements in fieldList - 1

    for row in cursor:
        

        # step through each field in the row by its index
        for index in fRange:
            if row[index] == None:
                row[index] = 0         #set null to zero
            
            
            cursor.updateRow(row)

print ("Finished")

Finished


### Reclassify Tau DEM flow direction to work with ESRI Flowaccumulation()

In [13]:
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start= datetime.datetime.now()
print ("Begin process", start)

arcpy.env.workspace = temp_dir
arcpy.env.overwriteOutput = True
spref = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.outputCoordinateSystem = spref


rastdesc2 = arcpy.Describe(flowdirrast)
print (rastdesc2.name)
rastname2 = rastdesc2.name
flowdircopy = Reclassify(flowdirrast, "Value", RemapValue([[1,1],[2,128],[3,64],[4,32],[5,16],[6,8],[7,4],[8,2]]))
flowdircopy.save(rastname2)
flowdirscribe = arcpy.Describe(flowdircopy)
flowdirpath = os.path.join(flowdirscribe.path, flowdirscribe.name)

stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)
print (flowdirpath)

Begin process 2020-02-06 14:27:01.518077
AnchStar_StrBrn_d8flowdir.tif
Process complete  0:00:02.863219
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\AnchStar_StrBrn_d8flowdir.tif


### Copy Integer DEM from TauDEM to local
 * outputs from TauDEM have Cellsize slightly smaller than original DEM source raster
   * This happened during the conversion of the orignal DEM from float to Integer
     

In [14]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.cellsize = flowdirpath

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

demdescribe = arcpy.Describe(dem)
demname = temp_dir + demdescribe.name
dem_copy = ExtractByMask(dem, flowdirpath)
dem_copy.save(demname)

print ("Processes Complete")

Processes Complete


### Extract stream src by reclassified flow dir
 * Probably unecessary but going to leave in for now to make sure all rasters are aligned

In [15]:
arcpy.env.workspace = temp_dir
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath

streamdescribe = arcpy.Describe(orig_streamrast)
streamname = temp_dir + "\\" + streamdescribe.name
streamrast = ExtractByMask(orig_streamrast, flowdirpath)
streamrast.save(streamname)

print ("Extraction Complete")

Extraction Complete


### Check rasters
 * Compare extent, cellsize, projection

In [16]:
rastlist = [streamrast, flowdirpath, dem_copy]
for raster in rastlist:
    desc = arcpy.Describe(raster)

    
    raster_min = arcpy.Raster(raster).minimum
    raster_max = arcpy.Raster(raster).maximum
    raster_mean = arcpy.Raster(raster).mean
    extent = arcpy.Raster(raster).extent
    
    print("Raster name:      %s" % desc.name)
    print("Projection:      %s" % desc.SpatialReference.name)
    print("Compression Type: %s" % desc.compressionType)
    print("Raster Format:    %s" % desc.format)
    print("Height: %d" % desc.height)
    print("Width:  %d" % desc.width)
    print("Cellsize:  %f" % desc.meanCellHeight)
    print("Integer Raster: %s" % desc.isInteger)
    print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
    print (extent)
    print ("")

Raster name:      AnchStar_StrBrn_src.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 6559
Width:  8047
Cellsize:  4.999583
Integer Raster: True
Raster stats: min = 0.00 max = 1.00 mean = 0.01
118738.668585699 1076858.79982803 158970.316695937 1109651.06774838 NaN NaN NaN NaN

Raster name:      AnchStar_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 6559
Width:  8047
Cellsize:  4.999583
Integer Raster: True
Raster stats: min = 1.00 max = 128.00 mean = 27.82
118738.668585699 1076858.79982803 158970.316695937 1109651.06774838 NaN NaN NaN NaN

Raster name:      Anchor_Attributes_06022020AnchStar_StrBrn_2Int.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 6559
Width:  8047
Cellsize:  4.999583
Integer Raster: True
Raster stats: min = 0.00 max = 621.00 mean = 271.84
118738.668585699 1076858.79982803 158970.316695937 1109651.06774

### Add Elevation data from DEM


In [17]:
from arcpy.sa import *
start = datetime.datetime.now()
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))
print ("")

tablename = region + "_DEM_Zonal_Table"

print ("Calculating zonal statistics")
print("")

stop = datetime.datetime.now()  
elapsed = stop - start  

print ('Time to complete = ',elapsed)
print("")

ztable = ZonalStatisticsAsTable(awcjoin, 'OBJECTID', dem_copy, tablename, 'DATA', 'MIN_MAX_MEAN')

print ("Joining statistics to RCAs")
print("")

arcpy.JoinField_management(awcjoin, 'OBJECTID', ztable, 'OBJECTID_1', ['MIN','MAX','MEAN'])


stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)


Start script: 2020-02-06 14:27:19

Calculating zonal statistics

Time to complete =  0:00:00.013963

Joining statistics to RCAs

Time to complete =  0:00:04.682639


In [18]:
import pandas as pd
pd.set_option('display.max_columns', None)  
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis import GIS
gis = GIS()
sdf = pd.DataFrame.spatial.from_featureclass(awcjoin)
sdf.head()


,OBJECTID,rca_id,HUC12,Name,K_r,K_s,K_p,CH_r,CH_s,CH_p,CO_r,CO_s,CO_p,P_r,P_s,P_p,S_r,S_s,S_p,MIN,MAX,MEAN,SHAPE
0,1,1.0,190203010802,Stariski Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,124,133,128.603251,"{'rings': [[[132492.52273674682, 1107831.21941..."
1,2,2.0,190203010802,Stariski Creek,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,243,306,278.116076,"{'rings': [[[139821.91195113584, 1104616.48718..."
2,3,3.0,190203010802,Stariski Creek,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,242,310,276.727113,"{'rings': [[[139186.96485139057, 1104301.51339..."
3,4,4.0,190203010802,Stariski Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,203,233,224.389034,"{'rings': [[[136582.18201148883, 1105151.44262..."
4,5,5.0,190203010802,Stariski Creek,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,173,195,183.016694,"{'rings': [[[135052.30944832042, 1105441.41850..."


## Identify outlets for each RCA
  * Shrink RCAs by 15 meters shift outlet slighty upstream in order to avoid any errors that may smaller RCAs shifted into larger systems
 * Identify RCAs that were missed in the first operation and shrink by 8 meters
 * Identify any remaining RCAs missed by the first 2 operations and identify outlet
### Merge results together and remove any duplicates

### Create Flow Accumulation

In [19]:
from datetime import datetime
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath 
arcpy.env.extent = flowdirpath

today = datetime.now()
# Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)

print(time_stamp)

import datetime
start = datetime.datetime.now()

print ("Begin process", start)
print("")

flowrastname = temp_dir + "\\" + region + "_flow_accumulation_" + time_stamp + ".tif"                

print ("Creating ",flowrastname)
print("")

flowacc = FlowAccumulation(flowdirpath,"",  'FLOAT', 'D8') # create flow acc from flow dir
flowacc.save(flowrastname)
flowaccdescribe = arcpy.Describe(flowacc)
flowaccpath = os.path.join(flowacc.path,flowacc.name)


stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)


06022020
Begin process 2020-02-06 14:27:27.389508

Creating  C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor_flow_accumulation_06022020.tif

Process complete  0:00:35.729024


### Shrink RCA by 15 meters
* Consider filtering out very small RCAs (those that would be dissolved with Inside Buffer) prior to running 

In [20]:
import datetime
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()
print ("Begin Process", start)

inrca = awcjoin
buffval = -15 #shrink polygons by 15 meters
rcadesc = arcpy.Describe(inrca)
buffabs = abs(buffval)
insidebuffname = rcadesc.name + "_InsideBuffer_" + str(buffabs)
shrinkrca = arcpy.Buffer_analysis(inrca,insidebuffname,buffval)

print(shrinkrca)
stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)

Begin Process 2020-02-06 14:28:03.139477
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\Anchor_rca_huc12_largeover_sj_InsideBuffer_15
Process complete  0:00:14.041848


### Identify and reclassify stream grid using TauDEM stream source grid and rca stream reaches

In [21]:
from datetime import datetime

arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


today = datetime.now()
# Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)
import datetime
start = datetime.datetime.now()

print ("Begin process", start)
print("")


outname = temp_dir + "\\" + region + "_rcastream_src_maskextract_" + time_stamp + ".tif"
streamrast2 = ExtractByMask(streamrast, reachcopy)
streamrast2.save(outname)

streamname = temp_dir + "\\" + region + "_streams_raster_" + time_stamp + ".tif"
#streamcon = Con((Con(IsNull(streamrast),0,1) + streamrast2),0,1,"VALUE < 1") #Identify stream network from the two input stream grids and reclassify as 1 for stream and 0 everywhere else
streamcon = Con(IsNull(streamrast2), 0, streamrast2) #Reclassify null values in extracted stream src grid as 0
streamcon.save(streamname)

print (streamcon)

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print("")

Begin process 2020-02-06 14:28:17.202269

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor_streams_raster_06022020.tif
Process complete  0:00:04.168900



### Identify outlets using stream accumulation grid, shrunken buffer, and zonal statistics
 

In [22]:
start = datetime.datetime.now()
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


print ("Creating stream flow accumulation raster")

streamflowname = temp_dir + "\\" + region + "_stream_accumulation.tif"                
streamflow = SetNull(streamcon,flowaccpath,"Value = 0") # Create flow accumulation raster along stream network only
streamflow.save(streamflowname)

print ("Creating max accumulation raster")

max_acczon = temp_dir +"//" + region + "_max_acc_zon_ALL.tif"
max_acc_zon = arcpy.sa.ZonalStatistics(shrinkrca, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon.save(max_acczon)

print (max_acc_zon, "Created")
print("")
print ("Identifying highest accumulation cell for each catchment")

max_accatch = temp_dir +"//" + region + "_max_acc_catch_ALL.tif"
max_acc_catch = Con(streamflow == max_acc_zon,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch.save(max_accatch)

print ("Catchment outlets Id'd")
print (max_acc_catch)

stop = datetime.datetime.now()
elapsed = stop - start  
print ('Time to complete = ',elapsed)


Creating stream flow accumulation raster
Creating max accumulation raster
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor_max_acc_zon_ALL.tif Created

Identifying highest accumulation cell for each catchment
Catchment outlets Id'd
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor_max_acc_catch_ALL.tif
Time to complete =  0:00:12.651217


### Convert to point
 * First set of outlets, some RCAs will be missed due to shrunken buffer

In [23]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ('Converting Max accumulation cells to points')
print ('')

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL"
outlets = arcpy.RasterToPoint_conversion(max_acc_catch,convers_name)


print ('Conversion COMPLETE')
print ('')

stop = datetime.datetime.now()
elapsed = stop - start

print ('Time to complete =',elapsed)

Converting Max accumulation cells to points

Conversion COMPLETE

Time to complete = 0:00:01.318479


### Identify missing RCAs
 * Select missed RCAs using outlets produced from the first operation
  * Rerun code above to create second outlet feature class using an 8 meter Inside Buffer
 


In [24]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ("Selecting RCAs missed during buffer shrink")
print("")

missedout = arcpy.SelectLayerByLocation_management(awcjoin, 'INTERSECT', outlets, 1, 'NEW_SELECTION', 'INVERT') #Identify missed RCAs

buffval = -8 #shrink missed polygons by 8 meters
rcadesc = arcpy.Describe(inrca)
buffabs = abs(buffval)
insidebuffname = rcadesc.name + "_InsideBuffer_" + str(buffabs)
shrinkrca2 = arcpy.Buffer_analysis(missedout, insidebuffname,buffval)

print ("Creating max accumulation raster")
print("")

max_acczon2 = temp_dir +"//" + region + "_max_acc_zon_ALL2.tif"
max_acc_zon2 = arcpy.sa.ZonalStatistics(shrinkrca2, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon2.save(max_acczon2)

print ("Identifying highest accumulation cell for each catchment")
print("")

max_accatch2 = temp_dir +"//" + region + "_max_acc_catch_ALL2.tif"
max_acc_catch2 = Con(streamflow == max_acc_zon2,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch2.save(max_accatch2)

print ("Catchment outlets Id'd")
print("")

print ('Converting Max accumulation cells to points')
print("")

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL2"
outlets2 = arcpy.RasterToPoint_conversion(max_acc_catch2,convers_name)

print ('Conversion COMPLETE')
print('')

print("Merging outlets and deleting any duplicates")
print ("")

pointmergename = outgdb + "\\" + region + "_outlets_Merge1"
outletmerge = arcpy.Merge_management([outlets,outlets2 ],pointmergename)

print('')
print ('Merge COMPLETE')

stop = datetime.datetime.now()
elapsed = stop - start
print ('Time to complete =',elapsed)

Selecting RCAs missed during buffer shrink

Creating max accumulation raster

Identifying highest accumulation cell for each catchment

Catchment outlets Id'd

Converting Max accumulation cells to points

Conversion COMPLETE

Merging outlets and deleting any duplicates


Merge COMPLETE
Time to complete = 0:00:12.985705


### Run a third time without inside buffer on any missed outlets
 * Tend to be very small RCAs near the confluence of larger systems

In [25]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


start = datetime.datetime.now()

print ("Selecting RCAs missed during buffer shrink")
print("")

missedout2 = arcpy.SelectLayerByLocation_management(awcjoin, 'INTERSECT', outletmerge, 1, 'NEW_SELECTION', 'INVERT') #Identify missed RCAs
miss2 = arcpy.GetCount_management(missedout2)
print ("final selection of " + str(miss2) + " missed RCAs")

print ("Creating max accumulation raster")
print("")

max_acczon3 = temp_dir +"//" + region + "_max_acc_zon_ALL3.tif"
max_acc_zon3 = arcpy.sa.ZonalStatistics(missedout2, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon3.save(max_acczon3)

print ("Identifying highest accumulation cell for each catchment")
print("")

max_accatch3 = temp_dir +"//" + region + "_max_acc_catch_ALL3.tif"
max_acc_catch3 = Con(streamflow == max_acc_zon3,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch3.save(max_accatch3)

print ("Catchment outlets Id'd")
print("")

print ('Converting Max accumulation cells to points')
print("")

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL3"
outlets3 = arcpy.RasterToPoint_conversion(max_acc_catch3,convers_name)

print ('Conversion COMPLETE')
print('')

print("Merging outlets")
print ("")

pointmergename2 = outgdb + "\\" + region + "_outlets_Merge2"
outletmerge2 = arcpy.Merge_management([outletmerge, outlets3],pointmergename2)

print('')
print ('Merge COMPLETE')

joinname = outgdb + "\\" + region + "_RCA_Outlets_" + time_stamp
idjoin2 = arcpy.SpatialJoin_analysis(outletmerge2, awcjoin, joinname, 'JOIN_ONE_TO_ONE', 'KEEP_ALL',"", 'INTERSECT')

print ("Join Complete...deleting duplicates")
print ("")

arcpy.DeleteIdentical_management(idjoin2,'SHAPE')

print ("Extracting flow accumulation to outlet")
print ("")

ExtractMultiValuesToPoints(idjoin2,[[streamflow, "Flow_Accumulation"]])

print ("Process complete")

count = arcpy.GetCount_management(idjoin2)

print (str(count) + " Outlets created")

stop = datetime.datetime.now()
elapsed = stop - start
print ('Time to complete =',elapsed)

Selecting RCAs missed during buffer shrink

final selection of 10 missed RCAs
Creating max accumulation raster

Identifying highest accumulation cell for each catchment

Catchment outlets Id'd

Converting Max accumulation cells to points

Conversion COMPLETE

Merging outlets


Merge COMPLETE
Join Complete...deleting duplicates

Extracting flow accumulation to outlet

Process complete
1211 Outlets created
Time to complete = 0:00:16.331894


### Calculate Mean Elevation for RCA + all upstream contributing area
 * Create elevation weighted flow accumulation and divide by flow accumulation
 * Extract to outlet

In [26]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ("Begin process", start)
print("")

elevw8name = temp_dir + "\\" + region + "_elevation_weighted_flowacc.tif"
elevw8flow = FlowAccumulation(flowdirpath, dem_copy, 'FLOAT', 'D8') # create glacially weighted flow accumulation
elevw8flow.save(elevw8name)

print(elevw8flow)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print ("")
print ("Calculate Mean Elevation along stream network")
print ("")

start = datetime.datetime.now()

meanelevname = temp_dir + "\\" + region + "stream_MEAN_elev.tif"                
meanelev = SetNull(streamflow,(elevw8flow / flowaccpath ),"Value = 0") #Limit output to stream network
meanelev.save(meanelevname)

ExtractMultiValuesToPoints(idjoin2,[[meanelev, "Mean_Elev_Contributing_Area"]])

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print(meanelev)


Begin process 2020-02-06 14:29:04.757200

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor_elevation_weighted_flowacc.tif

Process complete  0:00:37.548543

Calculate Mean Elevation along stream network

Process complete  0:00:10.724643
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchorstream_MEAN_elev.tif


### Create Final attributed RCA on local

In [27]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

rcadesc = arcpy.Describe(rcacopy)
name = rcadesc.name
rcacopyname = name +"_attributed_" + time_stamp
finalrca = arcpy.CopyFeatures_management(rcacopy, rcacopyname)

for field in arcpy.ListFields(finalrca):
    joindrop.append(field.name)
    
keepfields = ['OBJECTID', 'Shape', 'rca_id','Shape_Length', 'Shape_Area']
for field in keepfields:
    joindrop.remove(field)

arcpy.DeleteField_management(finalrca, joindrop)

tempdesc = arcpy.Describe(tempsites)
tempcopyname = name + "_temp_sites_attributed_" + time_stamp
tempfinal = arcpy.CopyFeatures_management(tempsites,tempcopyname)
ExtractMultiValuesToPoints(tempfinal,[[dem_copy, "Elevation_meters"]])

print("Extracting elevation at temperature site")
print(tempfinal)
print ("")

outletcopyname = region + "_rca_outlets_attributed_" + time_stamp
outcopy = arcpy.CopyFeatures_management(idjoin2, outletcopyname)
arcpy.DeleteIdentical_management(outcopy,'rca_id')
count = arcpy.GetCount_management(outcopy)

print (str(count) + " Outlets created")
print("")

count2 = arcpy.GetCount_management(finalrca)

print (str(count2) + " RCAs created in the " + str(region) + "study area")
print ("")

arcpy.JoinField_management(finalrca,"rca_id",outcopy, "rca_id")



Extracting elevation at temperature site
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\anchor_rcas_temp_sites_attributed_06022020

1210 Outlets created

1210 RCAs created in the Anchorstudy area



<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anchor_rcas_attributed_06022020'>

### Recalculate slope percent reaches
 * (rise/run*100)

In [28]:
from arcpy.sa import * 
method = "BILINEAR"
prop = "Z_MIN;Z_MAX;Z_MEAN;SURFACE_LENGTH"
perc_calc = '(!Z_Max!-!Z_Min!)/!SLength!*100'

print ("Adding Surface information to rca reaches")
print ("")

arcpy.ddd.AddSurfaceInformation(reachcopy,dem_copy,prop,method)

print ("Adding Surface info fields")
print ("")

arcpy.AddField_management(reachcopy,"reach_length", "DOUBLE","","","", "Reach Length in meters")
arcpy.AddField_management(reachcopy,"reach_slope", 'DOUBLE',"","","","Reach slope in percent")
arcpy.CalculateField_management(reachcopy,"reach_slope", perc_calc)
arcpy.CalculateField_management(reachcopy,"reach_length", '!SLength!')




Adding Surface information to rca reaches

Adding Surface info fields



<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anch_rca_reaches'>

## Calcute percent land cover metrics in 30meter buffered area

### Download NLCD
 * NLCD is currently the best available landcover dataset with coverage for the entire state

In [29]:
import datetime
wurl = r"https://prd-tnm.s3.amazonaws.com/StagedProducts/NLCD2011/Land_Cover/Alaska/ak_nlcd_2011_landcover_1_15_15.zip"
start = datetime.datetime.now() 
wname = wurl[-34:]
print ('Downloading',wname)

wzippath = str(ziploc) + '/'+ str(wname) #path to save download to plus name of download
# headers = {"Range": "bytes=0-100"}  # first 100 bytes
# rh= requests.get(wurl, headers)
# print(rh.status_code)
# print(rh.headers['content-type'])
# print(rh.encoding)
r = requests.get(wurl)
if not os.path.exists(wzippath):
    with open(wzippath,'wb') as f:
        f.write(r.content)
        
print('')
print ('ALL DOWNLOADS COMPLETE')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)


ALL DOWNLOADS COMPLETE
Time to complete =  0:00:29.009184


### Unzip NLCD
#### Must have 7zip installed and located in the path below, change if necessary


In [30]:
### Unzip landcover data

import subprocess
os.chdir(ziploc)
for dir in os.listdir():
    if "ak_nlcd_2011" in dir:
        wzip = os.path.abspath(dir)
        print ('Unzipping ', wzip)
        #wzip = zipfile.ZipFile(file_name) # create zipfile object
        #wzip.extractall(extractloc) # extract file to dir
        #wzip.close() # close file
        #os.remove(file_name) # delete zipped file if required
        wuz = subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + wzip + ' -o' + extractloc)
        print ('Finished extracting AK NLCD')
print('')
print ('Unzipping complete')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)

Unzipping  C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\zips\ak_nlcd_2011_landcover_1_15_15.zip
Finished extracting AK NLCD

Unzipping complete
Time to complete =  0:00:29.054064


### Download Glacier data - KENAI ONLY
 * approx 1 gb download as of 20200117

In [31]:
# import datetime
# wurl = r"http://www.glims.org/download/latest" #latest glims data
# start = datetime.datetime.now() 
# wname = "GLIMS_Data.zip"
# print ('Downloading',wname)


# wzippath = str(ziploc) + '/'+ str(wname) #path to save download to plus name of download
# # headers = {"Range": "bytes=0-100"}  # first 100 bytes
# # rh= requests.get(wurl, headers)
# # print(rh.status_code)
# # print(rh.headers['content-type'])
# # print(rh.encoding)
# r = requests.get(wurl)
# if not os.path.exists(wzippath):
#     with open(wzippath,'wb') as f:
#         f.write(r.content)
        
# print('')
# print ('ALL DOWNLOADS COMPLETE')
# stop = datetime.datetime.now()  
# elapsed = stop - start  
# print ('Time to complete = ',elapsed)

# os.chdir(ziploc)
# start = datetime.datetime.now()
# for item in os.listdir():
#     if "GLIMS" in item:
#         file_name = os.path.abspath(item) # get full path of files
#         zip_ref = zipfile.ZipFile(file_name) # create zipfile object
#         zip_ref.extractall(extractloc) # extract file to dir
#         zip_ref.close() # close file
#         #os.remove(file_name) # delete zipped file if required   
#         print ('Unzipping..', file_name)
#         print('')

# print ('Unzipping complete')
# stop = datetime.datetime.now()  
# elapsed = stop - start  
# print ('Time to complete = ',elapsed)

### Buffer by 30 meters and clip to study area
 * Buffers will overlap but this is not an issue
  * Adding text field to copy NHDPLUSID (Using Float field may cause issues with panda display) and add and calc shape area in square meters as a check

In [32]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

reachdesc = arcpy.Describe(reachcopy)
streambuffname = reachdesc.name + "_buff30"
riv_buff30 = arcpy.Buffer_analysis(reachcopy,streambuffname,30, 'FULL', 'FLAT', 'NONE')
arcpy.AddField_management(riv_buff30,"Buff_Area_Sqm", 'DOUBLE')
arcpy.CalculateField_management(riv_buff30, 'Buff_Area_Sqm','!shape.area@squaremeters!')


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\anch_rca_reaches_buff30'>

### Create Shrub and Forest Layers
 * Buffer study area by 60 meters to ensure no cells are missed in extraction
  * Extract NLCD raster by Study Watershed polygon mask and convert to Polygon

In [33]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.extent = flowdirpath
arcpy.env.snapRaster = flowdirpath

studybuffname = region + "_buff60"
study_buff = arcpy.Buffer_analysis(studycopy,studybuffname,60, 'FULL', 'ROUND', 'ALL')

print ("Watershed Buffer complete", study_buff)
print ("")

from arcpy.sa import *
os.chdir(extractloc)
arcpy.env.workspace = extractloc
raster_list = arcpy.ListRasters()
start = datetime.datetime.now()
for raster in raster_list:
    if "ak_nlcd_2011" in raster:
        
        print (raster)
        print ("")
        
        desc = arcpy.Describe(raster)
        name = desc.baseName
        outname = temp_dir + "\\" + name + "_extract.tif"
        
        print ("Extracting raster ", name)
        print ("")
        
        extract = ExtractByMask(raster, study_buff)
        extract.save(outname)
        
        print ("Extraction Complete")
        print (extract)
        print ("")
                
stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print ("")

arcpy.env.workspace = outgdb
nlcdpolyname = region + "_nlcd_poly"
nlcd_poly = arcpy.RasterToPolygon_conversion(extract,nlcdpolyname, 'NO_SIMPLIFY', 'Land_Cover')

print (nlcd_poly)

Watershed Buffer complete C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\Anchor_buff60

ak_nlcd_2011_landcover_1_15_15.img

Extracting raster  ak_nlcd_2011_landcover_1_15_15

Extraction Complete
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\ak_nlcd_2011_landcover_1_15_15_extract.tif

Process complete  0:00:00.614981

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\Anchor_nlcd_poly


### Add and calc area in sq meters as check

In [34]:
arcpy.AddField_management(nlcd_poly,"Area_Sqm", 'DOUBLE')
arcpy.CalculateField_management(nlcd_poly, 'Area_Sqm','!shape.area@squaremeters!')
arcpy.AddField_management(nlcd_poly,"Land_Cover_Reclass", 'TEXT')


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Anchor\\Anchor_attributes\\Anchor_Attributes_06022020\\Anchor.gdb\\Anchor_nlcd_poly'>

### Add field and reclass cover types using following dictionary
 * Use update cursor and dictionary to reclassify

In [35]:
start = datetime.datetime.now()

d = {'Barren Land':'Barren',
'Cultivated Crops':'Crops',
'Deciduous Forest':'Forest',
'Mixed Forest':'Forest',
'Evergreen Forest':'Forest',
'Grassland/Herbaceous':'Herbaceous',
'Sedge/Herbaceous':'Herbaceous',
'Developed, High Intensity':'High intensity',
'Developed, Low Intensity':'Low intensity',
'Developed, Medium Intensity':'Medium intensity',
'Developed, Open Space':'Open space',
'Pasture/Hay':'Pasture',
'Perennial Ice/Snow':'Perennial Ice',
'Shrub/Scrub':'Shrub',
'Dwarf Shrub':'Shrub',
'Open Water':'Water',
'Emergent Herbaceous Wetlands':'Wetlands',
'Woody Wetlands':'Wetlands'}

fields = ['Land_Cover','Land_Cover_Reclass']
with arcpy.da.UpdateCursor(nlcd_poly, (fields)) as rows:
    for row in rows:
        if row[0] not in d.keys():
            print ("{} not in list".format(row[0]))
        else:
            row[1] = d[row[0]]
            rows.updateRow(row)
    del row 
del rows

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)


Process complete  0:00:03.179511


### Tabulate intersection between 30m buffered river segments (defined by nhdplusid or reachid) and nlcd polygon created above
 
 * Table output of land cover percent and total area for each land cover type within the buffered river segment 

In [36]:

# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# tab_int = arcpy.TabulateIntersection_analysis(riv_buff30, identifierfield, nlcd_poly,tabtablename, ['Land_Cover','Land_Cover_Reclass'], 'Area_Sqm')

# print (tab_int)
# print ("")

# stop = datetime.datetime.now()
# elapsed = stop - start

# print ("Process complete ", elapsed)

In [37]:
start = datetime.datetime.now()
arcpy.env.workspace = outgdb
tab2name = "Reclassified_" + tabtablename
tab_int2 = arcpy.TabulateIntersection_analysis(riv_buff30, identifierfield, nlcd_poly,tab2name, ['Land_Cover_Reclass'], 'Area_Sqm')

print (tab_int2)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Anchor\Anchor_attributes\Anchor_Attributes_06022020\Anchor.gdb\Reclassified_anch_rca_reaches_NLCD_Tab_Int

Process complete  0:00:27.526089


### Create Identity and dissolve to check against tabulate intersection



In [38]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# riv_nlcd_id = arcpy.Identity_analysis(riv_buff30, nlcd_poly,identname, 'ALL')
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id)
# print ("")

# idcount = arcpy.GetCount_management(riv_nlcd_id)

# print (idcount)

### Dissolve Identity
 * Keep ID, landcover, reclass, and buffered area
 * Add/calculate area in sqm and percent cover

In [39]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# disname = identname + "_All_Land_Dissolve"

# print("Begin Dissolve", start)
# print("")

# riv_nlcd_i0.d_dis = arcpy.Dissolve_management(riv_nlcd_id,disname, [identifierfield,'Land_Cover','Land_Cover_Reclass','Buff_Area_Sqm'], '', 'MULTI_PART', 'DISSOLVE_LINES')
# arcpy.AddField_management(riv_nlcd_id_dis,"Percent_Cover", 'DOUBLE')
# arcpy.AddField_management(riv_nlcd_id_dis,'Shape_Area_Sqm', 'DOUBLE')
# arcpy.CalculateField_management(riv_nlcd_id_dis, 'Shape_Area_Sqm','!shape.area@squaremeters!')
# percent_calc = "!Shape_Area_Sqm!/!Buff_Area_Sqm!*100"
# arcpy.CalculateField_management(riv_nlcd_id_dis, 'Percent_Cover',percent_calc)
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id_dis)
# print ("")

# disscount = arcpy.GetCount_management(riv_nlcd_id_dis)

# print (disscount)


In [40]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# disname2 = identname + "_Land_Reclass_Dissolve"

# print("Begin Dissolve", start)
# print("")

# riv_nlcd_id_dis2 = arcpy.Dissolve_management(riv_nlcd_id,disname2, [identifierfield, 'Land_Cover_Reclass','Buff_Area_Sqm'], '', 'MULTI_PART', 'DISSOLVE_LINES')
# arcpy.AddField_management(riv_nlcd_id_dis2,"Percent_Cover", 'DOUBLE')
# arcpy.AddField_management(riv_nlcd_id_dis2,'Shape_Area_Sqm', 'DOUBLE')
# arcpy.CalculateField_management(riv_nlcd_id_dis2, 'Shape_Area_Sqm','!shape.area@squaremeters!')
# percent_calc = "!Shape_Area_Sqm!/!Buff_Area_Sqm!*100"
# arcpy.CalculateField_management(riv_nlcd_id_dis2, 'Percent_Cover',percent_calc)
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id_dis2)
# print ("")

# disscount = arcpy.GetCount_management(riv_nlcd_id_dis2)

# print (disscount)

### Convert table to numpy array

In [41]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
field_list = []
for field in arcpy.ListFields(tab_int2):
    field_list.append(field.name)
tabint_arr = arcpy.da.TableToNumPyArray(tab_int2,field_list)
df = pd.DataFrame(tabint_arr)
df

,OBJECTID,reachid,Land_Cover_Reclass,Area_Sqm,AREA,PERCENTAGE
0,1,1,Forest,14627.22,14627.22,26.55
1,2,1,Shrub,7029.21,7029.21,12.76
2,3,1,Wetlands,33426.76,33426.76,60.68
3,4,2,Forest,34313.47,34313.47,95.02
4,5,2,Shrub,1799.23,1799.23,4.98
...,...,...,...,...,...,...
2934,2935,1211,Forest,19691.05,19691.05,65.05
2935,2936,1211,Shrub,10476.18,10476.18,34.61
2936,2937,1211,Wetlands,103.10,103.10,0.34
2937,2938,1212,Forest,8402.58,8402.58,23.10


### List of unique land cover classes from tabulate intersection
 * All land cover types that are present in the study area

In [42]:
lancov = df['Land_Cover_Reclass'].unique()
print (sorted(lancov))

['Barren', 'Crops', 'Forest', 'Low intensity', 'Medium intensity', 'Open space', 'Pasture', 'Shrub', 'Water', 'Wetlands']


### Reshape dataframe and set index
 * Easier to read

In [43]:
df2 = df.drop(["OBJECTID","AREA"],axis=1)
df2 = df2.set_index([identifierfield,'Land_Cover_Reclass'])
df2


Area_Sqm  PERCENTAGE
reachid Land_Cover_Reclass                      
1       Forest              14627.22       26.55
        Shrub                7029.21       12.76
        Wetlands            33426.76       60.68
2       Forest              34313.47       95.02
        Shrub                1799.23        4.98
...                              ...         ...
1211    Forest              19691.05       65.05
        Shrub               10476.18       34.61
        Wetlands              103.10        0.34
1212    Forest               8402.58       23.10
        Shrub               27979.62       76.90

[2939 rows x 2 columns]

### Compare Dissolve to Table

In [44]:
# import pandas as pd
# pd.set_option('display.max_columns', None)  
# from arcgis.features import GeoAccessor, GeoSeriesAccessor
# from arcgis import GIS
# gis = GIS()
# sdf = pd.DataFrame.spatial.from_featureclass(riv_nlcd_id_dis2)
# sdf[[identifierfield,'Land_Cover_Reclass','Buff_Area_Sqm','Shape_Area_Sqm','Percent_Cover']].sort_values(by=[identifierfield])


### Pivot Tab Intersection table and join to rca reaches
 * Turn land cover classes stored in rows to fields

In [45]:
arcpy.env.workspace = outgdb
pivname = region + "Landcover_Pivot_Table"
tabpivot = arcpy.PivotTable_management(tab_int2,"reachid","Land_Cover_Reclass","PERCENTAGE",pivname)
arcpy.JoinField_management(reachcopy,"reachid",tabpivot,"reachid",["Forest","Shrub","Wetlands"])
arcpy.AddField_management(reachcopy,"Riparian", 'TEXT',"","",250,"Percent riparian (forest + shrub) cover in 30 meter buffer surrounding reach")

fieldList = ["Forest","Shrub","Wetlands"]
with arcpy.da.UpdateCursor(reachcopy, fieldList) as cursor:
        for row in cursor:
            Urow = row
            for i in range (len(fieldList)):
                if Urow[i] == None:
                    Urow[i] = 0
                
            cursor.updateRow(Urow)

print ("Processing complete")
ripcalc = "!Forest!+!Shrub!"
arcpy.CalculateField_management(reachcopy,"Riparian", ripcalc)

for field in arcpy.ListFields(reachcopy):
    print (field.name)



Processing complete
OBJECTID
Shape
reachid
Shape_Length
Z_Min
Z_Max
Z_Mean
SLength
reach_length
reach_slope
Forest
Shrub
Wetlands
Riparian


### Create Glacier layer and calculate percent cover by RCA - KENAI ONLY
 * Use <a href = "http://glims.colorado.edu/glacierdata/">GLIMS</a> for glacier data - may require manual download
  * Large dataset that needs to be converted to feature class, projected, and clipped to study area
    * Returned void poly error when initially clipped so these steps may resolve the issue.  Try repair geometry if it does not.
    * Create a raster layer with 1 for glacier and 0 for no-glacier. Use the flow accumulation tool to get additive value for upstream contributing area for each RCA and then divide by flow accumulation value for each RCA.

In [46]:
####add kenai glacier code here


### Rename Fields and delete unecessary joins
 * create dictionary to store oldname/new name alias combos

In [47]:
rcaDict = {'rca_id': ('rca_id', 'Unique ID associated with each reach contributing area (rca)'),
'HUC12': ('HUC12', '12th digit hydrologic unit code associated with rca'),
'Name': ('HUC_name', 'name of 12th digit hydrologic unit code associated with rca'),
'K_r': ('K_r', 'Presence/absence of Chinook salmon rearing habitat in rca'),
'K_s': ('K_s', 'Presence/absence of Chinook salmon spawning habitat in rca'),
'K_p': ('K_p', 'Presence/absence of Chinook salmon in rca'),
'CH_r': ('CH_r', 'Presence/absence of chum salmon rearing habitat in rca'),
'CH_s': ('CH_s', 'Presence/absence of chum salmon spawning habitat in rca'),
'CH_p': ('CH_p', 'Presence/absence of chum salmon in rca'),
'CO_r': ('CO_r', 'Presence/absence of coho salmon rearing habitat in rca'),
'CO_s': ('CO_s', 'Presence/absence of coho salmon spawning habitat in rca'),
'CO_p': ('CO_p', 'Presence/absence of coho salmon in rca'),
'P_r': ('P_r', 'Presence/absence of pink salmon rearing habitat in rca'),
'P_s': ('P_s', 'Presence/absence of pink salmon spawning habitat in rca'),
'P_p': ('P_p', 'Presence/absence of pink salmon in rca'),
'S_r': ('S_r', 'Presence/absence of sockeye salmon rearing habitat in rca'),
'S_s': ('S_s', 'Presence/absence of sockeye salmon spawning habitat in rca'),
'S_p': ('S_p', 'Presence/absence of sockeye salmon in rca'),
'MIN': ('rca_elev_min', 'Minimum rca elevation '),
'MAX': ('rca_elev_max', 'Maximum rca elevation '),
'MEAN': ('rca_elev_mn', 'Mean rca elevation '),
'Flow_Accumulation': ('flowacc', 'Number of upstream cells draining to rca pour point'),
'Mean_Elev_Contributing_Area': ('ca_elev_mn', 'Mean elevation of upstream contributing area draining to rca pour point')}

reachDict = {'reachid': ('reach_id', 'Unique ID associated with each confluence to confluence reach '), 'Forest': ('Forest', 'Percent forest cover in 30 meter buffer surrounding reach'),
                         'Shrub': ('Shrub', 'Percent shrub cover in 30 meter buffer surrounding reach'),
                         'Wetlands': ('Wetland', 'Percent wetland cover in 30 meter buffer surrounding reach')}
tempDict = {}
outletDict = {}

In [54]:
for field in arcpy.ListFields(reachcopy):
    keyval = field.name
    if keyval in reachDict:
        newname = reachDict[keyval][0]
        newalias = reachDict[keyval][1]
        
        print (newname, newalias)
        print ("")
        
        arcpy.AlterField_management(reachcopy, keyval, newname, newalias)

for field in arcpy.ListFields(finalrca):       
    keyval = field.name
    if keyval in rcaDict:
        newname = rcaDict[keyval][0]
        newalias = rcaDict[keyval][1]

        print (newname, newalias)
        print("")

        arcpy.AlterField_management(finalrca, keyval, newname, newalias)


reach_id Unique ID associated with each confluence to confluence reach 

Forest Percent forest cover in 30 meter buffer surrounding reach

Shrub Percent shrub cover in 30 meter buffer surrounding reach

Wetland Percent wetland cover in 30 meter buffer surrounding reach

rca_id Unique ID associated with each reach contributing area (rca)

HUC12 12th digit hydrologic unit code associated with rca

HUC_name name of 12th digit hydrologic unit code associated with rca

K_r Presence/absence of Chinook salmon rearing habitat in rca

K_s Presence/absence of Chinook salmon spawning habitat in rca

K_p Presence/absence of Chinook salmon in rca

CH_r Presence/absence of chum salmon rearing habitat in rca

CH_s Presence/absence of chum salmon spawning habitat in rca

CH_p Presence/absence of chum salmon in rca

CO_r Presence/absence of coho salmon rearing habitat in rca

CO_s Presence/absence of coho salmon spawning habitat in rca

CO_p Presence/absence of coho salmon in rca

P_r Presence/absence 

### Delete unecessary fields



In [55]:
rcadrops = ["Join_Count","TARGET_FID","pointid","grid_code","rca_id_1"]
reachdrops =["Z_Min","Z_Max","Z_Mean","SLength"] 

for field in rcadrops:
    arcpy.DeleteField_management(finalrca,field)

for field in reachdrops:
    arcpy.DeleteField_management(reachcopy,field)

for field in arcpy.ListFields(reachcopy):
    
    print (field.name)
    
print ("")

for field in arcpy.ListFields(finalrca):
    
    print (field.name)

OBJECTID
Shape
reach_id
Shape_Length
reach_length
reach_slope
Forest
Shrub
Wetland
Riparian

OBJECTID
Shape
rca_id
Shape_Length
Shape_Area
HUC12
HUC_name
K_r
K_s
K_p
CH_r
CH_s
CH_p
CO_r
CO_s
CO_p
P_r
P_s
P_p
S_r
S_s
S_p
rca_elev_min
rca_elev_max
rca_elev_mn
flowacc
ca_elev_mn


### Copy to Network GDB

In [60]:
arcpy.env.workspace = networkgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
spref = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

exportfcs= []
exportfcs = [tempfinal,finalrca,outcopy]

for fc in exportfcs:
    desc = arcpy.Describe(fc)
    arcpy.CopyFeatures_management(fc,desc.name)
reachdesc = arcpy.Describe(reachcopy)

reachcopyname = reachdesc.name + "_attributed_" + str(time_stamp)
arcpy.CopyFeatures_management(reachcopy,reachcopyname)

print ("copy complete")    

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 002852: Output Feature Class: T:\\Aquatic\\KFHP\\Geodatabases\\Anchor.gdb\anchor_rcas_temp_sites_attributed_06022020 exists within geodatabase as T:\Aquatic\KFHP\Geodatabases\Anchor.gdb\Watersheds\anchor_rcas_temp_sites_attributed_06022020
Failed to execute (CopyFeatures).
